In [19]:
import numpy as np
import scipy as sp

import plotly.plotly as ply
import plotly.graph_objs as go

import scipy.fftpack as fft
import siggen
import onestage

In [3]:
#input parameters
a = siggen.Antenna()
a.load("/home/dima/.landing/AFS32.ini")
f0 = 120e6
N = 2 ** 16
fs = f0 * N / (N/2 -2);
f_res = fs / N
df = f_res
beam_width = onestage.beam_width_est(antenna=a, f0=f0, df=df, fs=fs, f_res=f_res)
print(np.rad2deg(beam_width))

[ 29.4  13.5]


In [14]:
size = np.array([np.pi // beam_width[1] * 3, 2 * np.pi // beam_width[0] ** 3], np.int)
lh = onestage.arp(a, f0, df, fs, f_res, size)

In [18]:
alpha = np.linspace(0, 2 * np.pi, size[1])
betta = np.linspace(-np.pi / 2 , np.pi / 2, size[0])
alpha_grid, betta_grid = np.meshgrid(alpha, betta)

trace = go.Surface(    x=np.rad2deg(alpha_grid),
                       y=np.rad2deg(betta_grid),
                       z=lh)
data = [trace]
ply.iplot(data, filename='Antenna radiaton pattern Cartesian')

In [ ]:
max_ind = np.argmax(lh)
betta_max = max_ind % lh.shape[0]
alpha_max = max_ind % lh.shape[1]

sp.optimize.minimize(onestage.aoa_likelihood)